## **Recurrent Neural Networks (RNNs)**
---

## **Objective**

To understand the concepts of Recurrent Neural Networks (RNNs), their role in sequence modeling, the mathematical intuition behind their computations, the Backpropagation Through Time (BPTT) algorithm, and their practical implementation using TensorFlow/Keras.

---

## **Overview**

Recurrent Neural Networks (RNNs) are a class of neural networks designed specifically for sequence data. They are widely used in applications such as:
- Text generation
- Machine translation
- Time-series forecasting
- Speech recognition

RNNs can capture temporal dependencies by maintaining a "hidden state" that carries information across time steps. This ability to retain context is what makes them ideal for sequential tasks.

---

## **Key Features of RNNs**
1. **Sequential Data Processing**: RNNs process data sequentially, making them suitable for variable-length inputs.
2. **Shared Parameters**: RNNs use the same weights for each time step, reducing the model complexity.
3. **Hidden State**: A hidden state stores information from previous time steps to influence the current step's output.

---

## **Mathematical Foundations of RNNs**

1. **Hidden State Update**:
   At each time step `t`, the hidden state is calculated as:


h_t = f(W_hh * h_(t-1) + W_xh * x_t + b_h)

where:
- `h_t`: Hidden state at time `t`
- `h_(t-1)`: Hidden state from the previous time step
- `x_t`: Input at time `t`
- `W_hh`: Weight matrix for hidden state
- `W_xh`: Weight matrix for input
- `b_h`: Bias vector
- `f`: Activation function (e.g., tanh or ReLU)

2. **Output Calculation**:
The output at time `t` is given by:


y_t = g(W_ho * h_t + b_o)

where:
- `y_t`: Output at time `t`
- `W_ho`: Weight matrix for output
- `b_o`: Bias vector for output
- `g`: Activation function (e.g., softmax for classification tasks)

---

## **Backpropagation Through Time (BPTT)**

1. **Loss Function**:
For a sequence of length `T`, the total loss is:

dW = sum(dL_t / dW)

Note: Long sequences can cause vanishing or exploding gradients.

---

## **Advantages and Disadvantages of RNNs**

### **Advantages**:
1. Handles sequential and temporal data effectively.
2. Shares parameters across time steps, reducing model size.

### **Disadvantages**:
1. Struggles with long-term dependencies due to vanishing gradients.
2. Computationally expensive for long sequences.
3. Difficulty parallelizing computations.

---

## **Step 1: Import Libraries**


In [3]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

### **Explanation of Libraries**
1. `numpy`: For numerical computations.
2. `tensorflow.keras`: For building the RNN model.
3. `Tokenizer` and `pad_sequences`: For text preprocessing.
4. `matplotlib.pyplot`: For visualizing training metrics.

---

## **Step 2: Dataset Overview**

For this example, we will use a simple text dataset to demonstrate how RNNs can generate sequences. The dataset consists of a short string of text.

---

## **Dataset Example**


In [4]:
text = "hello world"
vocab = sorted(set(text))  # Get unique characters
char_to_index = {char: i for i, char in enumerate(vocab)}
index_to_char = {i: char for i, char in enumerate(vocab)}

print("Vocabulary:", vocab)
print("Character to Index Mapping:", char_to_index)

Vocabulary: [' ', 'd', 'e', 'h', 'l', 'o', 'r', 'w']
Character to Index Mapping: {' ': 0, 'd': 1, 'e': 2, 'h': 3, 'l': 4, 'o': 5, 'r': 6, 'w': 7}


### **Explanation**
1. `sorted(set(text))`: Extracts unique characters from the text and sorts them.
2. `char_to_index`: Maps each character to a unique index.
3. `index_to_char`: Maps each index back to its corresponding character.

---

## **Step 3: Sequence Creation**

The text will be split into sequences of fixed length. Each sequence will be mapped to a numerical representation.
---

In [5]:
sequence_length = 5
sequences = []

for i in range(len(text) - sequence_length):
    seq = text[i:i + sequence_length]
    sequences.append([char_to_index[char] for char in seq])

sequences = np.array(sequences)
print("Sequences:", sequences)


Sequences: [[3 2 4 4 5]
 [2 4 4 5 0]
 [4 4 5 0 7]
 [4 5 0 7 5]
 [5 0 7 5 6]
 [0 7 5 6 4]]


### **Explanation**
1. `sequence_length`: Defines the length of each sequence.
2. A loop is used to extract overlapping sequences of the specified length from the text.
3. Each character in the sequence is converted into its corresponding index using `char_to_index`.

---

## **Step 4: Model Definition**

We will define an RNN model using TensorFlow/Keras.
---


In [6]:
model = Sequential([
    Embedding(input_dim=len(vocab), output_dim=8, input_length=sequence_length),
    SimpleRNN(16, return_sequences=False),
    Dense(len(vocab), activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### **Explanation of the Model**
1. **Embedding Layer**:
   - Converts each character index into a dense vector of fixed size (8 in this case).
2. **SimpleRNN Layer**:
   - Processes the sequence of embeddings and produces a hidden state of size 16.
3. **Dense Layer**:
   - Produces an output for each class (vocabulary size) using the softmax activation function.
4. **Loss and Optimizer**:
   - Loss: Sparse categorical crossentropy for multi-class classification.
   - Optimizer: Adam optimizer for efficient training.

---

## **Step 5: Training the Model**

The model will be trained on the sequences for a few epochs.

---


In [7]:

X = sequences[:, :-1]
y = sequences[:, -1]

history = model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 2.0803
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.1667 - loss: 2.0710
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.3333 - loss: 2.0617
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3333 - loss: 2.0525
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.3333 - loss: 2.0433
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3333 - loss: 2.0341
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5000 - loss: 2.0248
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5000 - loss: 2.0154
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6667 - loss: 2.0059
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.6667 - loss: 1.9962
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8333 - loss: 1.9864
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6667 -

### **Explanation**
1. `X`: Contains the input sequences (all but the last character of each sequence).
2. `y`: Contains the target character (last character of each sequence).
3. `epochs`: Number of times the model will see the entire dataset during training.

---

## **Step 6: Generating Text**

After training, we will use the model to generate text by predicting the next character.

---

In [8]:
def generate_text(model, start_string, num_generate):
    input_seq = [char_to_index[char] for char in start_string]
    input_seq = np.expand_dims(input_seq, axis=0)

    text_generated = start_string

    for _ in range(num_generate):
        predictions = model.predict(input_seq)
        next_char_index = np.argmax(predictions)
        next_char = index_to_char[next_char_index]

        text_generated += next_char
        input_seq = np.expand_dims([next_char_index], axis=0)

    return text_generated

generated_text = generate_text(model, "hello", 10)
print("Generated Text:", generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Generated Text: hello wrl wrl w


### **Explanation**
1. The function `generate_text` takes a starting string and generates the specified number of characters.
2. For each step:
   - The model predicts probabilities for the next character.
   - The character with the highest probability is chosen as the next character.
   - The input sequence is updated, and the process repeats.

---

## **Conclusion**

This notebook introduced Recurrent Neural Networks (RNNs) and demonstrated their use in sequence modeling tasks. We explored:
- RNN concepts and mathematics.
- Backpropagation Through Time (BPTT).
- Building and training an RNN for text generation.
---


## **Additional Concepts and Enhancements for RNNs**

### **Theoretical Concepts**

1. **RNN Variants**:
   - Vanilla RNNs suffer from issues like vanishing gradients, which make it hard to model long-term dependencies in sequences.
   - **LSTMs (Long Short-Term Memory)** and **GRUs (Gated Recurrent Units)** are advanced architectures designed to handle these limitations by introducing gating mechanisms.
   - Key differences:
     - **LSTMs**: Use input, output, and forget gates to control the flow of information.
     - **GRUs**: Simplify the LSTM architecture with fewer gates (update and reset gates) but still achieve competitive performance.

2. **Bidirectional RNNs**:
   - These networks process sequences in both forward and backward directions, which is particularly useful for tasks like text translation and speech recognition, where both past and future context matter.

3. **Attention Mechanism**:
   - Attention allows the model to focus on specific parts of the input sequence that are most relevant to the task.
   - Introduced in seq2seq models, attention is a precursor to Transformer-based architectures like BERT and GPT.

4. **Exploding Gradients**:
   - While vanishing gradients occur due to small updates in weights, **exploding gradients** happen when the gradients become too large.
   - Mitigation:
     - Use techniques like **gradient clipping** to scale down large gradients during backpropagation.

5. **Applications of RNNs**:
   - RNNs are widely used for:
     - **Natural Language Processing (NLP)**: Sentiment analysis, language modeling, text generation, and translation.
     - **Time-Series Analysis**: Predicting stock prices, weather forecasting, and energy demand.
     - **Speech Recognition**: Transcribing spoken words into text.

---

### **Mathematics**

1. **Backpropagation Through Time (BPTT)**:
   - RNNs unroll over time to process sequences. Gradients are computed for each time step and aggregated.
   - The challenge:
     - As gradients are multiplied at each step, they can become very small (vanishing gradients) or very large (exploding gradients).
   - **Steps**:
     - Compute the forward pass for all time steps.
     - Unroll the network for `T` time steps.
     - Calculate gradients of loss with respect to weights and biases over all time steps.

---

### **Practical Enhancements**

1. **Hyperparameter Tuning**:
   - Important parameters for RNNs:
     - **Hidden state size**: Controls the capacity of the model. Larger sizes capture more patterns but are computationally expensive.
     - **Sequence length**: Determines how many time steps the model processes at once. Longer sequences can capture more context but increase memory usage.
     - **Learning rate**: Affects the speed of convergence during training.

2. **Evaluation Metrics**:
   - For text generation models, **perplexity** is often used as a metric to measure how well a model predicts a sequence. Lower perplexity indicates better performance.

3. **Regularization**:
   - Overfitting is common in RNNs, especially on small datasets.
   - Use techniques like **dropout** to randomly "drop" connections between layers during training.

4. **Using Pretrained Models**:
   - While RNNs are effective, modern tasks often leverage pretrained Transformer models like BERT and GPT, which outperform RNNs on many NLP benchmarks.
   - However, RNNs remain relevant for smaller datasets and certain time-series tasks.

